# axondeepseg: ometiff-tests for ADS/ivadomed needs
#### Tools:
#### Tifffile (Python package)
#### libvips (C Foreign Function Interface (CFFI) for Python)

In [ ]:
import sys
import tifffile
import numpy as np
from matplotlib import pyplot as plt
from ctypes import *
cdll.LoadLibrary("C:/Users/eti_m/Documents/UK-Biobank/vips-dev-8.11/bin/libgobject-2.0-0.dll")  # Change for the path where vips-dev-8.11 is
cdll.LoadLibrary("C:/Users/eti_m/Documents/UK-Biobank/vips-dev-8.11/bin/libvips-42.dll")        # idem
import pyvips

## 1) Read OME-TIFF image + metadata

Context: Here, we open an OME-TIFF file using the Python package *tifffile*. We use an OME-TIFF sample data from OME that can be found here: https://docs.openmicroscopy.org/ome-model/5.6.3/ome-tiff/data.html

In [ ]:
with tifffile.TiffFile('tubhiswt_C0.ome.tif', is_ome=True) as tif:
	s1 = tif.series[0].asarray()                            # Read series 1 (highest-res) from OME-TIFF as a numpy array
	meta = tif.ome_metadata                                 # Read OME metadata as a string
print("Series 1 type:", type(s1))
print("Series 1 shape:", str(s1.shape))
print("Series 1 dims: (channels, time points, xdim, ydim)")

# Display of the different time points of the channels (not necessary)
fig, axs = plt.subplots(nrows = s1.shape[0], ncols = s1.shape[1])
fig.suptitle('Time points of the channels of the OME-TIFF file')
[axi.set_axis_off() for axi in axs.ravel()]
for t in range(s1.shape[1]):
    for c in range(s1.shape[0]):
        axs[c,t].imshow(s1[c,t,:,:], cmap='gray')
fig.set_size_inches(20, 2*s1.shape[0])
plt.show()

# Convertion XML string to dict
meta_dict = tifffile.xml2dict(meta)['OME']
print("meta_dict:", meta_dict)

# In this example's OME-TIFF, the metadata is organized in different sub-dicts in meta_dict:
#print("\n\n Experiment:", meta_dict['Experiment'])
#print("\n\n Experimenter:", meta_dict['Experimenter'])
#print("\n\n Instrument:", meta_dict['Instrument'])
#print("\n\n Image:", meta_dict['Image'])
#print("\n\n Creator:", meta_dict['Creator'])
#print("\n\n UUID:", meta_dict['UUID'])

# If we want to acces one specific field:
print("\n 'DimensionOrder' in the sub dict 'Pixels' and 'Image':", meta_dict['Image']['Pixels']['DimensionOrder'])

We notice a 'DimensionOrder'='XYZTC' and a shape '(C, T, Y, X)'. As we can see in the class *TiffFile* of the file *tifffile.py*, the shape (as well as the parameter 'axes') is reversed compared to 'DimensionOrder'. Because 'SizeZ'=1, the dimension doesn't appear in the shape, though it should be (2, 20, 1, 512, 512).

## 2) Write OME-TIFF image + metadata

Context: We also want to be able to write predictions (numpy arrays) as OME-TIFF. A random numpy array is generated and, using the Python package *tifffile*, the image is incorporated in a OME-TIFF file with some specific metadata fields.

In [ ]:
# Generating the random numpy array (dimension sizes are arbitrary)
data = np.random.randn(1,10,3,72,108).astype('uint8')

# Display of the different time points of the channels (not necessary)
fig, axs = plt.subplots(nrows = data.shape[2], ncols = data.shape[1])
fig.suptitle('Time points of the channels of the random numpy array')
[axi.set_axis_off() for axi in axs.ravel()]
for t in range(data.shape[1]):
    for c in range(data.shape[2]):
        axs[c,t].imshow(data[0,t,c,:,:], cmap='gray')
fig.set_size_inches(20, 2*data.shape[2])
plt.show()

# Here are the metadata fields that we can add in the parameter 'metadata={ }' to the OME-TIFF with tifffile:
#
# Under 'Image' and 'Pixels':
# - Name, AcquisitionDate, Description, PhysicalSizeX, PhysicalSizeXUnit, PhysicalSizeY, PhysicalSizeYUnit, PhysicalSizeZ, PhysicalSizeZUnit,
#   TimeIncrement, TimeIncrementUnit.
#
# Under 'Image', 'Pixels' and 'Plane':
# - DeltaTUnit, ExposureTime, ExposureTimeUnit, PositionX, PositionXUnit, PositionY, PositionYUnit, PositionZ, PositionZUnit.
# 
# Under 'Image', 'Pixels' and 'Channel':
# - Name, AcquisitionMode, Color, ContrastMethod, EmissionWavelength, EmissionWavelengthUnit, ExcitationWavelength, ExcitationWavelengthUnit,
#   Fluor, IlluminationType, NDFilter, PinholeSize, PinholeSizeUnit, PockelCellSetting.

# When writing with tifffile, we have to specify the shape of the array in the parameter 'axes'. As the shape is (1,10,3,72,108) = (Z,T,C,Y,X), the parameter 'axes'='ZTCYX'. The metadata field "DimensionOrder" will be reversed, i.e. XYCTZ.
tifffile.imwrite('ometiff-tests_random.ome.tif', data, compress=7, metadata={'axes': 'ZTCYX',
                                                                    'Description':'Just a test OME-TIFF with random data.',
                                                                    'PhysicalSizeX':'1',
                                                                    'PhysicalSizeXUnit':'um',
                                                                    'PhysicalSizeY':'2',
                                                                    'PhysicalSizeYUnit':'um',
                                                                    'PhysicalSizeZ':'0.5',      # Arbitrary values
                                                                    'PhysicalSizeZUnit':'um'})

In [ ]:
#   From the class COMPRESSION in the tifffile.py file, here are the possible values of the compression parameter.
#   While writing, we give to the parameter 'compress' the value related to the type of compression we want:
#
#            NONE = 1  # Uncompressed
#            CCITTRLE = 2  # CCITT 1D
#            CCITT_T4 = 3  # 'T4/Group 3 Fax',
#            CCITT_T6 = 4  # 'T6/Group 4 Fax',
#            LZW = 5
#            OJPEG = 6  # old-style JPEG
#            JPEG = 7
#            ADOBE_DEFLATE = 8
#            JBIG_BW = 9
#            JBIG_COLOR = 10
#            JPEG_99 = 99
#            KODAK_262 = 262
#            NEXT = 32766
#            SONY_ARW = 32767
#            PACKED_RAW = 32769
#            SAMSUNG_SRW = 32770
#            CCIRLEW = 32771
#            SAMSUNG_SRW2 = 32772
#            PACKBITS = 32773
#            THUNDERSCAN = 32809
#            IT8CTPAD = 32895
#            IT8LW = 32896
#            IT8MP = 32897
#            IT8BL = 32898
#            PIXARFILM = 32908
#            PIXARLOG = 32909
#            DEFLATE = 32946
#            DCS = 32947
#            APERIO_JP2000_YCBC = 33003  # Leica Aperio
#            JPEG_2000_LOSSY = 33004  # BioFormats
#            APERIO_JP2000_RGB = 33005  # Leica Aperio
#            ALT_JPEG = 33007  # BioFormats
#            JBIG = 34661
#            SGILOG = 34676
#            SGILOG24 = 34677
#            JPEG2000 = 34712
#            NIKON_NEF = 34713
#            JBIG2 = 34715
#            MDI_BINARY = 34718  # Microsoft Document Imaging
#            MDI_PROGRESSIVE = 34719  # Microsoft Document Imaging
#            MDI_VECTOR = 34720  # Microsoft Document Imaging
#            LERC = 34887  # ESRI Lerc
#            JPEG_LOSSY = 34892
#            LZMA = 34925
#            ZSTD_DEPRECATED = 34926
#            WEBP_DEPRECATED = 34927
#            PNG = 34933  # Objective Pathology Services
#            JPEGXR = 34934  # Objective Pathology Services
#            ZSTD = 50000
#            WEBP = 50001
#            JPEGXL = 50002  # JXL
#            PIXTIFF = 50013

In [ ]:
# Validation of the metadata dictionnary
with tifffile.TiffFile('ometiff-tests_random.ome.tif', is_ome=True) as tif:
	meta_random = tif.ome_metadata                                 # Read OME metadata as a string
meta_dict_random = tifffile.xml2dict(meta_random)['OME']
print("meta_dict:", meta_dict_random)

## 3) Convert different type files to OME-TIFF with metadata

### 3.1) TIFF to OME-TIFF

Context: We have an existing PNG file that we want to convert to a TIF file and finally in the OME-TIFF format using the Python package *pyvips*.

In [ ]:
# PNG to TIF
png = pyvips.Image.new_from_file('77.png')
png.tiffsave('77.tif')

# TIF to OME-TIFF
tif = pyvips.Image.new_from_file('77.tif')

## Adding an alpha channel
#if tif.hasalpha():
#    tif = tif[:-1]

## TIFF copy and metadata setting
tif = tif.copy()
tif.set_type(pyvips.GValue.gint_type, "page-height", tif.height)
tif.set_type(pyvips.GValue.gstr_type, "image-description",
f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{tif.bands}"
                SizeT="1"
                SizeX="{tif.width}"
                SizeY="{tif.height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")
## Write data and metadata with pyvips
tif.tiffsave('ometiff-tests_tif.ome.tif', compression="jpeg", pyramid=True)

### 3.2) NDPI to OME-TIFF
Context: We have an existing multiresolution NDPI file that we want to convert in the OME-TIFF format. The multiresolution NDPI contains metadata that we want to read with *tifffile* and transfer to the OME-TIFF as well using *pyvips*.

#### 3.2.1) Read the NDPI file of interest

In [ ]:
with tifffile.TiffFile('test3-TRITC_2_(560).ndpi', is_ndpi=True) as ndpi:

    # Read data with tifffile (only to display the NDPI image. If not required, the following lines can be removed)
    series1 = ndpi.pages[0].asarray()
    print("Data shape:", str(series1.shape), "\nData type:", type(series1), "\nSeries 3:")
    plt.imshow(ndpi.pages[2].asarray())                        # Display of Series 3 (lower resolution) to save time
    plt.show()

    # Read metadata with tifffile
    meta = ndpi.pages[0].ndpi_tags
    print("\nMetadata type:", type(meta), "\nMetadata:", meta)

In [ ]:
# Extract metadata fields of interest
print("'Make' field:\t\t", meta['Make'])
print("'Model' field:\t\t", meta['Model'])
print("'Software' field:\t", meta['Software'])

#### 3.2.2) Generate the OMETIFF with the NDPI data

In [ ]:
# Read with pyvips
ndpi = pyvips.Image.new_from_file('test3-TRITC_2_(560).ndpi')

# Adding an alpha channel
if ndpi.hasalpha():
    ndpi = ndpi[:-1]

# NDPI copy and metadata setting
ndpi = ndpi.copy()
ndpi.set_type(pyvips.GValue.gint_type, "page-height", ndpi.height)
ndpi.set_type(pyvips.GValue.gstr_type, "image-description",
f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Contributors>
    </Contributors>
    <Dataset>
        <technicalInfo>
                SoftwareVersions="{meta['Software']}"
        </technicalInfo>
    </Dataset>
    <Funders>
    </Funders>
    <Image ID="Image:0">
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{ndpi.bands}"
                SizeT="1"
                SizeX="{ndpi.width}"
                SizeY="{ndpi.height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
    <Instrument 
            Make="{meta['Make']}"
            Model="{meta['Model']}">
    </Instrument>
    <Publications>
    </Publications>
    <Specimen>
    </Specimen>
</OME>""")

# Write data and metadata with pyvips
ndpi.tiffsave('ometiff-tests_ndpi.ome.tif', compression="jpeg", tile=True, tile_width=512, tile_height=512, pyramid=True)

### (3.2.3) Validation by reading the generated OME-TIFF)

In [ ]:
# Validation of the dimensions and the output image
with tifffile.TiffFile('ometiff-tests_ndpi.ome.tif', is_ome=True) as test:
    print("Data shape:", str(test.pages[0].shape),"= (Y,X,C)", "\nData type:", type(test.pages[0].asarray()), "\nSeries 6:")
    plt.imshow(test.pages[4].asarray())
    plt.show()
    test_meta_dict = tifffile.xml2dict(test.ome_metadata)['OME']

print("\n'DimensionOrder' in the sub dict 'Pixels' and 'Image':", test_meta_dict['Image']['Pixels']['DimensionOrder'])
print("'SizeX' in the sub dict 'Pixels' and 'Image':", test_meta_dict['Image']['Pixels']['SizeX'])
print("'SizeY' in the sub dict 'Pixels' and 'Image':", test_meta_dict['Image']['Pixels']['SizeY'])
print("'SizeC' in the sub dict 'Pixels' and 'Image':", test_meta_dict['Image']['Pixels']['SizeC'])
print("'SizeZ' in the sub dict 'Pixels' and 'Image':", test_meta_dict['Image']['Pixels']['SizeZ'])
print("'SizeT' in the sub dict 'Pixels' and 'Image':", test_meta_dict['Image']['Pixels']['SizeT'])


In [ ]:
# Validation of the metadata dictionnary
print(test_meta_dict)